# data prep

In [5]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('../data/gradient_descent_implementation_data.csv')

In [6]:
admissions

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [8]:
# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,0
396,0,560,3.04,0,0,1,0
397,0,460,2.63,0,1,0,0
398,0,700,3.65,0,1,0,0


In [10]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

In [14]:
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
...,...,...,...,...,...,...,...
395,0,0.279614,1.603135,0,1,0,0
396,0,-0.239793,-0.919418,0,0,1,0
397,0,-1.105469,-1.996759,0,1,0,0
398,0,0.972155,0.683454,0,1,0,0


In [18]:
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)


In [19]:
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,0,-0.066657,0.289305,0,1,0,0
280,0,0.625884,1.445476,0,1,0,0
33,1,1.837832,1.603135,0,0,1,0
210,0,1.318426,-0.131120,0,0,0,1
93,0,-0.066657,-1.208461,0,1,0,0
...,...,...,...,...,...,...,...
393,1,0.279614,0.946220,0,1,0,0
134,0,-0.239793,-1.155908,0,1,0,0
306,1,-0.412928,-0.577822,1,0,0,0
383,0,0.625884,1.603135,1,0,0,0


In [20]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

# gradient descent

In [21]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [26]:
def sigmoid_prime(x):
    """
    # Derivative of the sigmoid function
    """
    return sigmoid(x) * (1 - sigmoid(x))

In [22]:
# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use the same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape


In [27]:
last_loss = None

In [28]:
# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5


In [31]:
features

,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,-0.066657,0.289305,0,1,0,0
280,0.625884,1.445476,0,1,0,0
33,1.837832,1.603135,0,0,1,0
210,1.318426,-0.131120,0,0,0,1
93,-0.066657,-1.208461,0,1,0,0
...,...,...,...,...,...,...
393,0.279614,0.946220,0,1,0,0
134,-0.239793,-1.155908,0,1,0,0
306,-0.412928,-0.577822,1,0,0,0
383,0.625884,1.603135,1,0,0,0


In [41]:
for e in range(epochs):
#     Return a new array of given shape and type, filled with zeros.
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(x, weights))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error * output * (1 - output)
        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term*x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records
    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Train loss:  nan
Prediction accuracy: 0.750


<ipython-input-41-21081eacc87b>:38: RuntimeWarning: invalid value encountered in greater
  predictions = tes_out > 0.5
